## training the model

In [1]:
import os

In [2]:
%pwd

'c:\\Users\\DELL\\ML project\\CNNClassification\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'c:\\Users\\DELL\\ML project\\CNNClassification'

In [5]:
cd "C:\Users\DELL\ML project\CNNClassification"

c:\Users\DELL\ML project\CNNClassification


In [6]:
%pwd

'c:\\Users\\DELL\\ML project\\CNNClassification'

In [7]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class TrainingConfig:
    root_dir: Path
    trained_model_path: Path
    updated_base_model_path: Path
    training_data: Path
    params_epochs: int
    params_batch_size: int
    params_is_augmentation: bool
    params_image_size: list


@dataclass(frozen=True)
class PrepareCallbacksConfig:
    root_dir: Path
    tensorboard_root_log_dir: Path
    checkpoint_model_filepath: Path

In [8]:
from src.CNNClassification.constants import *
from src.CNNClassification.utils import read_yaml, create_directories
import tensorflow as tf

[2025-10-20 20:48:36,984: INFO: utils: NumExpr defaulting to 4 threads.]


In [9]:
class ConfigurationManager:
    def __init__(
        self, 
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

    
    def get_prepare_callback_config(self) -> PrepareCallbacksConfig:
        config = self.config.prepare_callbacks
        model_ckpt_dir = os.path.dirname(config.checkpoint_model_filepath)
        create_directories([
            Path(model_ckpt_dir),
            Path(config.tensorboard_root_log_dir)
        ])

        prepare_callback_config = PrepareCallbacksConfig(
            root_dir=Path(config.root_dir),
            tensorboard_root_log_dir=Path(config.tensorboard_root_log_dir),
            checkpoint_model_filepath=Path(config.checkpoint_model_filepath)
        )

        return prepare_callback_config

    def get_training_config(self) -> TrainingConfig:
        training = self.config.training
        prepare_base_model = self.config.prepare_base_model
        params = self.params
        training_data = os.path.join(self.config.data_ingestion.unzip_dir, "PetImages")
        create_directories([
            Path(training.root_dir)
        ])

        training_config = TrainingConfig(
            root_dir=Path(training.root_dir),
            trained_model_path=Path(training.trained_model_path),
            updated_base_model_path=Path(prepare_base_model.updated_base_model_path),
            training_data=Path(training_data),
            params_epochs=params.EPOCHS,
            params_batch_size=params.BATCH_SIZE,
            params_is_augmentation=params.AUGMENTATION,
            params_image_size=params.IMAGE_SIZE
        )

        return training_config

In [10]:
import time

In [11]:
class PrepareCallback:
    def __init__(self, config: PrepareCallbacksConfig):
        self.config = config

    
    @property
    def _create_tb_callbacks(self):
        timestamp = time.strftime("%Y-%m-%d-%H-%M-%S")
        tb_running_log_dir = os.path.join(
            self.config.tensorboard_root_log_dir,
            f"tb_logs_at_{timestamp}",
        )
        return tf.keras.callbacks.TensorBoard(log_dir=tb_running_log_dir)
    

    @property
    def _create_ckpt_callbacks(self):
        return tf.keras.callbacks.ModelCheckpoint(
            filepath=self.config.checkpoint_model_filepath,
            save_best_only=True
        )


    def get_tb_ckpt_callbacks(self):
        return [
            self._create_tb_callbacks,
            self._create_ckpt_callbacks
        ]

In [15]:
import os
import urllib.request as request
from zipfile import ZipFile
import tensorflow as tf
#from tensorflow.keras.optimizers.legacy import SGD
import numpy as np
import time

In [16]:
import tensorflow as tf

# Place these two lines near the top of your script/notebook:
tf.config.run_functions_eagerly(True) 
tf.data.experimental.enable_debug_mode() # Optional, but can help

In [31]:
import tensorflow as tf
from pathlib import Path

# NOTE: This assumes 'TrainingConfig' and necessary imports are available.

class Training:
    def __init__(self, config):
        self.config = config
        self.model = None
        self.train_generator = None
        self.valid_generator = None
    
    # 1. MODIFICATION: Load Model without old optimizer state (Fixes ValueError)
    def get_base_model(self):
        """Loads the pre-trained model architecture and weights, discarding the old optimizer state."""
        self.model = tf.keras.models.load_model(
            self.config.updated_base_model_path,
            compile=False  # <--- CRITICAL: Prevents loading the saved optimizer state
        )
    
    # 2. ADDITION: Compile Model with a new optimizer (Fixes 'Must call compile()' error)
    def prepare_full_model(self):
        """Creates a NEW optimizer instance and compiles the model."""
        
        # Ensure learning rate is defined (Fixes NameError)
        try:
            learning_rate = self.config.params_learning_rate 
        except AttributeError:
            # Fallback if config is misnamed
            print("Warning: 'params_learning_rate' not found in config. Using default 0.001.")
            learning_rate = 0.001 
        
        # Instantiate a brand NEW optimizer object (Fixes ValueError)
        new_optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate) 
        
        # Compile the model
        self.model.compile(
            optimizer=new_optimizer,  
            loss=tf.keras.losses.CategoricalCrossentropy(),
            metrics=["accuracy"]
        )
        
    # 3. train_valid_generator remains the same (using ImageDataGenerator)
    def train_valid_generator(self):
        """Sets up the training and validation data generators."""
        
        datagenerator_kwargs = dict(
            rescale = 1./255,
            validation_split=0.20
        )
        # ... (rest of the train_valid_generator code remains the same) ...
        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )
        
        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        print("Creating validation generator...")
        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )

        if self.config.params_is_augmentation:
            train_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
                rotation_range=40,
                horizontal_flip=True,
                width_shift_range=0.2,
                height_shift_range=0.2,
                shear_range=0.2,
                zoom_range=0.2,
                **datagenerator_kwargs
            )
        else:
            train_datagenerator = valid_datagenerator
            
        print("Creating training generator...")
        self.train_generator = train_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="training",
            shuffle=True,
            **dataflow_kwargs
        )

    # 4. save_model remains the same
    @staticmethod
    def save_model(path: Path, model: tf.keras.Model):
        model.save(path)


    # 5. train method remains the same
    def train(self, callback_list: list):
        """Fits the model to the training data."""
        self.steps_per_epoch = self.train_generator.samples // self.train_generator.batch_size
        self.validation_steps = self.valid_generator.samples // self.valid_generator.batch_size

        self.model.fit(
            self.train_generator,
            epochs=self.config.params_epochs,
            steps_per_epoch=self.steps_per_epoch,
            validation_steps=self.validation_steps,
            validation_data=self.valid_generator,
            callbacks=callback_list
        )

        self.save_model(
            path=self.config.trained_model_path,
            model=self.model
        )







# Assuming 'training' object is initialized and 'callback_list' is defined



In [ ]:
import tensorflow as tf
from pathlib import Path

# IMPORTANT: You must ensure Eager Execution is enabled at the start of your script/notebook:
# tf.config.run_functions_eagerly(True)

# NOTE: This assumes 'TrainingConfig' and other necessary imports are available.
# Replace {YOUR_CUSTOM_OBJECTS} with your actual dictionary if you have custom code.
# Example: {"CustomMetric": CustomMetricClass}
CUSTOM_OBJECTS = {} # If your model uses no custom code, leave this as {}

class Training:
    def __init__(self, config):
        self.config = config
        self.model = None
        self.train_generator = None
        self.valid_generator = None
    
    # 1. FIX: Load Model correctly (Optimizer & Callback fix)
    def get_base_model(self):
        """Loads the pre-trained model architecture and weights, discarding old optimizer state."""
        print(f"Loading base model from: {self.config.updated_base_model_path}")
        self.model = tf.keras.models.load_model(
            self.config.updated_base_model_path,
            custom_objects=CUSTOM_OBJECTS, # <--- FIX for pyfunc_XX error
            compile=False                  # <--- FIX for optimizer ValueError
        )
        print("Base model loaded successfully.")
    
    # 2. ADDITION/FIX: Compile Model with a new optimizer
    def prepare_full_model(self):
        """Creates a NEW optimizer instance and compiles the model."""
        
        # Define the learning rate (Fixes NameError/AttributeError if config is correct)
        try:
            learning_rate = self.config.params_learning_rate 
        except AttributeError:
            print("Warning: 'params_learning_rate' not found in config. Using default 0.001.")
            learning_rate = 0.001 
        
        # Instantiate a brand NEW optimizer object 
        new_optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate) 
        
        # Compile the model
        print("Compiling model with a fresh Adam optimizer...")
        self.model.compile(
            optimizer=new_optimizer,  
            loss=tf.keras.losses.CategoricalCrossentropy(),
            metrics=["accuracy"]
        )
        print("Model compiled successfully.")

    # 3. Data Generators (no changes needed here)
    def train_valid_generator(self):
        """Sets up the training and validation data generators."""
        
        datagenerator_kwargs = dict(
            rescale = 1./255,
            validation_split=0.20
        )

        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(**datagenerator_kwargs)
        
        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )

        if self.config.params_is_augmentation:
            train_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
                rotation_range=40,
                horizontal_flip=True,
                width_shift_range=0.2,
                height_shift_range=0.2,
                shear_range=0.2,
                zoom_range=0.2,
                **datagenerator_kwargs
            )
        else:
            train_datagenerator = valid_datagenerator

        self.train_generator = train_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="training",
            shuffle=True,
            **dataflow_kwargs
        )

    # 4. save_model (no changes needed)
    @staticmethod
    def save_model(path: Path, model: tf.keras.Model):
        print(f"Saving trained model to: {path}")
        model.save(path)
        print("Model saved.")


    # 5. train method (no changes needed)
    def train(self, callback_list: list):
        """Fits the model to the training data."""
        
        # Calculate steps (Fixes AttributeError if generators are not None)
        self.steps_per_epoch = self.train_generator.samples // self.train_generator.batch_size
        self.validation_steps = self.valid_generator.samples // self.valid_generator.batch_size
        
        print("Starting model training...")
        self.model.fit(
            self.train_generator,
            epochs=self.config.params_epochs,
            steps_per_epoch=self.steps_per_epoch,
            validation_steps=self.validation_steps,
            validation_data=self.valid_generator,
            callbacks=callback_list
        )
        print("Training finished.")

        self.save_model(
            path=self.config.trained_model_path,
            model=self.model
        )

Creating validation generator...
Found 78 images belonging to 2 classes.
Creating training generator...
Found 318 images belonging to 2 classes.


c:\Users\DELL\anaconda3\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


 1/19 ━━━━━━━━━━━━━━━━━━━━ 7:17 24s/step - accuracy: 0.3750 - loss: 0.8365

InvalidArgumentError: {{function_node __wrapped__IteratorGetNext_output_types_2_device_/job:localhost/replica:0/task:0/device:CPU:0}} ValueError: Could not find callback with key=pyfunc_39 in the registry.
Traceback (most recent call last):

  File "c:\Users\DELL\anaconda3\Lib\site-packages\tensorflow\python\ops\script_ops.py", line 257, in __call__
    raise ValueError(f"Could not find callback with key={token} in the "
                     "registry.")

ValueError: Could not find callback with key=pyfunc_39 in the registry.


	 [[{{node EagerPyFunc}}]] [Op:IteratorGetNext] name: 